In [2]:
import sqlite3 as sq3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
#%matplotlib inline

In [3]:
DBNAME = "Gold_Country_OSM2.db"
FETCH_ONE = 0
FETCH_ALL = 1
PHYSICAL_FEATURE_TAGS = ['shop','amenity','waterway','power','aerialway','man_made', \
                       'boundary','highway','barrier','leisure','historic','tourism',\
                       'building','traffic_calming','natural','aeroway','place', \
                       'railway','landuse','city','emergency','manhole','office']

## convert the PHYSICAL_FEATURE_TAGS list into a string for use in SQL queries
pfs = "'" + PHYSICAL_FEATURE_TAGS[0] + "'"
for tag in PHYSICAL_FEATURE_TAGS[1:]:
    pfs = pfs + ",'" + tag + "'"

In [4]:
## function to run one SQLite query and return either one row, or all of the result set
def run_sql(SQL, dbname=DBNAME, fetch=FETCH_ALL, params = None):
    with sq3.connect(dbname) as conn:
        cur = conn.cursor()
        if params == None:
            cur.execute(SQL)
        else:
            cur.execute(SQL, params)
        if fetch == FETCH_ALL:
            data = cur.fetchall()
            return data
        elif fetch == FETCH_ONE:
            data = cur.fetchone()
            return data

In [5]:
# Repeat count of nodes, ways, relations, and users.
TABLE_NAMES = ['nodes','ways','relations','users']
for table_name in TABLE_NAMES:
    tally = run_sql("SELECT COUNT(*) FROM " + table_name +"", fetch=FETCH_ONE)
    print "There are", tally[0], table_name,"in the database."

There are 622831 nodes in the database.
There are 42604 ways in the database.
There are 174 relations in the database.
There are 458 users in the database.


In [45]:
# What fraction of tagged ways had identified tagging errors:
SQL = "SELECT COUNT (DISTINCT id) FROM Way_Tags;"
Total_Ways = run_sql(SQL, fetch=FETCH_ONE)[0]
print Total_Ways, "ways with tags."
SQL = "SELECT COUNT (DISTINCT id) FROM Way_Tags WHERE key='error_corrected';"
Error_Ways = run_sql(SQL, fetch=FETCH_ONE)[0]
print Error_Ways, "with with identified tagging errors."
Tag_Error_Percent = float(Error_Ways) * 100 / Total_Ways
print "Approximately %2.2f percent of ways have identified tagging errors." % Tag_Error_Percent

## Many of the TIGER ways be 'junk' data also.  As far as I can tell they would need to be reviewed
## by on-the-ground mapping to know if they even really exist.
SQL = "SELECT COUNT(DISTINCT id) from Way_tags WHERE type = 'tiger' AND id NOT IN \
    (SELECT DISTINCT id from Way_tags WHERE key = 'name') AND id IN \
    (SELECT DISTINCT id from Way_tags WHERE key='highway' AND value='residential')"
No_name_TIGER_Ways = run_sql(SQL, fetch=FETCH_ONE)[0]
print No_name_TIGER_Ways, "no-name, TIGER-upload ways tagged as highway=residential."
print "Assuming half of these are erroneous in some way, the error rate in the ways table"
Total_Error_Percent = float(Error_Ways + No_name_TIGER_Ways/2) * 100 / Total_Ways
print "could be as high as %2.2f percent." % Total_Error_Percent

41898 ways with tags.
5713 with with identified tagging errors.
Approximately 13.64 percent of ways have identified tagging errors.
2160 no-name, TIGER-upload ways tagged as highway=residential.
Assuming half of these are erroneous in some way, the error rate in the ways table
could be as high as 16.21 percent.


In [46]:
# How many nodes are not part of a way, and should therefore indicate, independently, a primary
# map feature?  All of the nodes I identified as being incorrectly, or incompletely tagged were
# from this group.
SQL = "SELECT COUNT(*) from Nodes WHERE Nodes.id NOT IN \
    (SELECT DISTINCT node_id FROM Way_Nodes)"
Not_Way_Nodes = run_sql(SQL, fetch=FETCH_ONE)[0]
print "There are", Not_Way_Nodes, "nodes in the database which are not part of a way."
# How many nodes, not part of a way, were identified to have tagging errors?
SQL = "SELECT COUNT(Nodes.id) FROM Nodes JOIN Node_Tags on nodes.id = node_tags.id WHERE \
    node_tags.key='error_corrected' AND Nodes.id NOT IN \
    (SELECT DISTINCT node_id from Way_nodes)"
Tag_Error_Nodes = run_sql(SQL, fetch=FETCH_ONE)[0]
print Tag_Error_Nodes, "nodes were found to have tagging errors (all non-way nodes)."
print "For a tagging error rate of %2.2f percent." % (float(Tag_Error_Nodes) *100/Not_Way_Nodes)

There are 3627 nodes in the database which are not part of a way.
255 nodes were found to have tagging errors (all non-way nodes).
For a tagging error rate of 7.03 percent.


In [47]:
## How many different users in the Ways table, and how many elements did each contribute?
## Get result for the entire Ways table, then split into ways from the TIGER upload and those
## which are not tagged as being from TIGER dataset.
all_user_data = run_sql("SELECT users.user, COUNT (ways.id) as way_ct FROM Users JOIN Ways \
    ON Users.uid = Ways.uid GROUP BY user \
    ORDER BY way_ct DESC", fetch = FETCH_ALL)
tiger_user_data = run_sql("SELECT users.user, COUNT(DISTINCT ways.id) as way_ct FROM Users JOIN \
    Ways ON Users.uid = Ways.uid JOIN \
    Way_tags on ways.id = way_tags.id WHERE Way_tags.type = 'tiger' \
    GROUP BY user ORDER BY way_ct DESC", fetch = FETCH_ALL)
non_tiger_user_data = run_sql("SELECT users.user, COUNT(Ways.id) as Way_Ct FROM Users JOIN Ways \
    ON Users.uid = Ways.uid \
    WHERE Ways.id NOT IN (SELECT DISTINCT ways.id FROM Ways JOIN \
    Way_tags on ways.id = way_tags.id WHERE Way_tags.type = 'tiger') \
    GROUP BY USER ORDER BY Way_Ct DESC", fetch=FETCH_ALL)

df_all = pd.DataFrame.from_records(all_user_data,columns=['User_Name','w_All'], \
                                   index='User_Name')
df_tiger = pd.DataFrame.from_records(tiger_user_data,columns=['User_Name','w_Tiger'], \
                                     index='User_Name')
df_non = pd.DataFrame.from_records(non_tiger_user_data, columns=['User_Name','w_NonT'],\
                                   index='User_Name')
df_list = [df_all,df_tiger, df_non]
df_total = pd.concat(df_list,axis=1)
df_total.sort_values(by=('w_All'), ascending=False, inplace=True)
df_modes = df_total.mode()
df_total.fillna(value=0, inplace=True)
print df_total.head(10)
print
print "Mode for each column:"
print df_modes

                  w_All  w_Tiger  w_NonT
leni2016           6748      8.0  6740.0
balrog-kun         6248   6242.0     6.0
T99                3324    768.0  2556.0
DaveHansenTiger    3213   3213.0     0.0
nmixter            2578     83.0  2495.0
Charles_Smothers   1973    740.0  1233.0
RaquelFish09       1782      0.0  1782.0
AndrewSnow         1372   1121.0   251.0
tjstansell         1188     46.0  1142.0
CFish_007          1154      4.0  1150.0

Mode for each column:
   w_All  w_Tiger  w_NonT
0      1      1.0     1.0


In [48]:
df_total[['w_All','w_Tiger','w_NonT']].plot.hist(bins = 50, range=(0,100), title='Number of Edits by User:All Ways')
plt.show()

This histogram has the same shape regardless of range of values and whether all data, or the subgroups plotted.  There's a suggestion of a bump around 10-20 edits, but given the small numbers hard to say this looks significant.  In all cases the largest group of users is those with only one edit!  

In [49]:
## Examine user data from the Nodes table.  Limit analysis to nodes which are not part of ways.

Node_user_data = run_sql("SELECT users.user, COUNT(Nodes.id) as Node_ct from Nodes JOIN Users ON \
    Nodes.uid = Users.uid WHERE \
    Nodes.id NOT IN (SELECT DISTINCT node_id from Way_Nodes) GROUP BY user \
    ORDER BY Node_ct DESC", fetch=FETCH_ALL)

print len(Node_user_data)


154


In [50]:
df_node_user = pd.DataFrame.from_records(Node_user_data, columns=['User_Name','Edit_Count'], \
                                        index='User_Name')
print df_node_user.head()
print
print "Mode is:"
print df_node_user.mode()

           Edit_Count
User_Name            
T99              1451
iandees           400
amillar           318
nmixter           196
davidearl         184

Mode is:
   Edit_Count
0           1


In [51]:
df_node_user.plot.hist (bins=50, range=(0,100),title="Nodes Not In Ways: Edit Count by User")
plt.show()

In [52]:
## Count the number of times each tag in the PRIMARY_FEATURE_TAGS (pfs) list is used in the Nodes
## table for nodes which are not part of a way.
SQL = "SELECT Node_tags.key, COUNT (*) FROM Node_Tags WHERE Node_tags.key IN (" + pfs + ") AND \
    Node_tags.id NOT IN (SELECT DISTINCT node_id from Way_Nodes) \
    GROUP BY Node_Tags.key;"
data = run_sql(SQL, fetch=FETCH_ALL)
for datum in data:
    print datum[0], ":", datum[1]

aeroway : 23
amenity : 807
barrier : 36
building : 992
city : 89
emergency : 20
highway : 102
historic : 15
landuse : 306
leisure : 99
man_made : 90
manhole : 9
natural : 227
office : 8
place : 420
power : 72
railway : 21
shop : 160
tourism : 107
traffic_calming : 2
waterway : 88


In [53]:
# Look more closely at the features which probably represent building or other 
# sites of interest: amenity, building, historic, leisure, man_made, office, 
# railway, shop, and tourism.
SELECT_TAGS = ['amenity', 'building', 'historic','leisure','office', \
              'railway','shop','man_made','tourism']
SQL = "SELECT value, COUNT(*) from Node_Tags WHERE key = ?  GROUP BY value"
Node_tag_values = []
for tag in SELECT_TAGS:
    #print tag, ":"
    data = run_sql(SQL, params=(tag,), fetch=FETCH_ALL)
    ct = 0
    for datum in data:
        #print "\t", datum[0], ":", datum[1]
        ct += datum[1]
    #print ct, "total node tags of this type."
    #print "Total", len(data),  "different '"+tag+"' types.\n"
    Node_tag_values.append({'Tag_Name':tag,'Nodes':ct,'Node_Types':len(data)})

df_node_tv = pd.DataFrame(Node_tag_values)
df_node_tv.set_index(['Tag_Name'], inplace=True)

SQL = "SELECT value, COUNT(*) from Way_Tags WHERE key = ?  GROUP BY value"
Way_tag_values = []
for tag in SELECT_TAGS:
    #print tag, ":"
    data = run_sql(SQL, params=(tag,), fetch=FETCH_ALL)
    ct = 0
    for datum in data:
        #print "\t", datum[0], ":", datum[1]
        ct += datum[1]
    #print ct, "total way tags of this type."
    #print "Total", len(data),  "different '"+tag+"' types.\n"
    Way_tag_values.append({'Tag_Name':tag,'Ways':ct,'Way_Types':len(data)})

df_way_tv = pd.DataFrame(Way_tag_values)
df_way_tv.set_index(['Tag_Name'], inplace=True)

frames = [df_node_tv, df_way_tv]
df_all_tv = pd.concat(frames, axis=1)

cols = ['Nodes','Node_Types','Ways','Way_Types']
df_all_tv = df_all_tv[cols]

df_all_tv['Total'] = df_all_tv['Nodes'] + df_all_tv['Ways']
print df_all_tv

          Nodes  Node_Types  Ways  Way_Types  Total
Tag_Name                                           
amenity     824          53   499         32   1323
building    995          10  5859         18   6854
historic     15           6    10          2     25
leisure     106           8  1499         15   1605
office        8           4     1          1      9
railway     130           7   158          8    288
shop        161          52    29         13    190
man_made     93          13  2181         11   2274
tourism     111           9    16         11    127
